# Tarea 3 Cadenas de Markov


Autores: 
- Daniel Alejandro García Hernández
- David Camilo Cortes Salazar

En este notebook se encuentra una implementación de algunos resultados vistos en clase para el Gibbs Sampler aplicado a Hard-core y q-colorings. 

Las librerías necesarias para ejectuar el código son:

In [1]:
import numpy as np
import random
from pprint import pprint

# Modelo de Ising y Algoritmo de Propp - Wilson

### Muestras del modelo de Ising con inverso de temperatura usando MCMC

Iniciamos creando la grilla del módelo. Para esto, creamos una matriz 2x2 apoyandonos en la libreria Numpy y la declaramos de tamaño $kxk$. las dimensiones del grafo. $10\leq k \leq 20$.

Tambien creamos una lista con los beetas que vamos a evaluar y otra con los diferentes numeros de pasos con los que ejecturemos el algoritmo.

In [33]:
k = 10
beta_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
steps_list = [10**3, 10**4, 10**5]
muestras = 100

Ahora recordemos como se calculaba la energia del sistema dada una configuracion. Esto esta dado por la funcion $H(\eta)$ descrita acontinuacion:

$$H(\eta)=-\sum_{x\thicksim y}\eta_{x}\eta_{y}$$

Donde $\eta$ es la configuracion actual del sistema y $\eta_{x}$, $\eta_{y}$ son la posicion de dos nodos de $\eta$ tal que $x$ y $y$ estan conectados.

Con esto en mente creamos una funcion que calcula la energia del sistema dada una configuracion $\eta$ del modelo.

In [34]:
def calcular_energia(G, k):
    # Calcula la energia de la configuracion dada
    energia = 0
    for i in range(k):
        for j in range(k):
            if i != k-1:
                energia += G[i,j]*G[i+1, j]
            if j != k-1:
                energia += G[i,j]*G[i, j+1]
    return -1 * energia

Ahora usemos un gibbs sampler para implementar el metodo de montecarlo en nuestro problema. Este funcionara de la siguiente manera:

- Seleccione un vertice al azar $(\~{x}, \~{y})$ de la configurtacion actual $\eta$.
- Cambie el valor del vertice escogido (Si el vertice es 1, cambie por -1 y viceversa). Esto para obtener una nueva configuracion $\hat{\eta}$ del sistema.
- Calcule la energia de las configuraciones $\eta$ y $\hat{\eta}$.
- Calcule la probabilidad de cambio de $\eta$ y $\hat{\eta}$ utilizando la probabilidad de distribucion del vetice $(\~{x}, \~{y})$: 

$$ \frac{\exp^{2\beta (H(\hat{\eta})-H(\eta))}}{\exp^{2\beta (H(\hat{\eta})-H(\eta))} + 1} $$

- Genere un numero aleatorio uniforme $p$ entre 0 y 1
- Si $p < \Delta \pi$, acepta la configuracion $\hat{\eta}$. Si no, mantenemos con la configuracion $\eta$.
- Repetimos el algoritmo hasta completar las iteraciones propuestas.

Note que al calcular la energia de los sistemas no es necesario hacer todas las pusibles multiplicaciones de spines conectados. Esta energia se usara el el calculo de la probabilidad de cambio al hallar el valor: $H(\hat{\eta}) - H(\eta)$. Pero estas configuraciones solo difiren en un punto, luego tenemos lo siguiente:

$$H(\hat{\eta}) - H(\eta)= -\sum_{x\thicksim y}\hat{\eta}_{x}\hat{\eta}_{y} - (-\sum_{x\thicksim y}\eta_{x}\eta_{y}) = \sum_{x\thicksim y}\eta_{x}\eta_{y} - \sum_{x\thicksim y}\hat{\eta}_{x}\hat{\eta}_{y}$$

Y haciendo una particion entre los productos afectados por el nodo $(\~{x}, \~{y})$ donde $X = \{(x,y) : x\thicksim y \wedge x\not= \~x \wedge y\not= \~y\}$ y $\~X = \{(x,y) : x\thicksim y \wedge x = \~x \lor y = \~y\}$:

$$ = \sum_{X} \eta_{x}\eta_{y} - \hat{\eta}_{x}\hat{\eta}_{y} + \sum_{\~X} \eta_{x}\eta_{y} - \hat{\eta}_{x}\hat{\eta}_{y} = \sum_{\~X} \eta_{x}\eta_{y} - \hat{\eta}_{x}\hat{\eta}_{y} = -\sum_{\~X}\hat{\eta}_{x}\hat{\eta}_{y} - (-\sum_{\~X}\eta_{x}\eta_{y})$$

Pero como la unica diferencia entre $\hat{\eta}$ y $\eta$ el el signo en el nodo $(\~x, \~y)$, tenemos que $\sum_{\~X}\hat{\eta}_{x}\hat{\eta}_{y} = (-\sum_{\~X}\eta_{x}\eta_{y})$. Obteniendo:



$$= -(-\sum_{\~X}\eta_{x}\eta_{y}) - (-\sum_{\~X}\eta_{x}\eta_{y}) = \sum_{\~X}\eta_{x}\eta_{y} + \sum_{\~X}\eta_{x}\eta_{y} = 2\sum_{\~X}\eta_{x}\eta_{y}$$

Obteniendo una simplificacion en el numero de operaciones a realizar para hacer la diferencia de calores. En resumen, este resultado nos muestra que basta con hallar la diferencia del producto de los nodos que estan relacionados con el punto escogido de manera aleatoria.

A continuacion, creamos una nueva funcion para el calculo de la energia:

In [35]:
def calcular_diferencia_energia(G, k, x, y):
    # Calcula la diferencia de la energia en la configuracion dada
    energia = 0
    if x != 0:
        energia += G[x-1, y]
    if x != k-1:
        energia += G[x+1, y]
    if y != 0:
        energia += G[x, y-1]
    if y != k-1:
        energia += G[x, y+1]
    return 2 * G[x, y] * energia

Y con esto creamos el Gibbs sampler para nuestro modelo:

In [43]:
def aceptar_cambio(G, x, y, beta, energia):
    # Calcula la probabilidad de cambio de la nueva configuracion
    e = np.exp(2 * beta * energia)
    prob_cambio = e / (e+1)

    # Condicion de Cambio
    if np.random.rand() < prob_cambio: # Aplica Monte Carlo
        #G[x, y] *= -1 # Cambia el spin
        G[x, y] = 1 # Cambia el spin
    else: G[x, y] = -1 # Cambia el spin


def gibbs_sampler_mcmc(G, beta, iteraciones, k):
    for _ in range(iteraciones):
        
        # Selecciona spin aleatorio
        x = random.randint(0,k-1)
        y = random.randint(0,k-1)

        # Calcula la energia de la configuracion actual y siguiente segun el spin dado

        diferencia_energia = calcular_diferencia_energia(G, k, x, y)
        
        # Evalua el cambio
        aceptar_cambio(G, x, y, beta, diferencia_energia)
    
    return G

Ahora solo nos queda probar el algoritmo con los valores de beta y pasos del gibbs sampler que definimos al inicio para obtener el numero de muestras deseadas

In [37]:
configuraciones_mcmc = {}

# Loop principal
for beta in beta_list: # Beta a utilizar
    configuraciones_mcmc[beta] = {key: [] for key in steps_list}
    for step in steps_list[1:2]: # Numero de pasos a utilizar
        for _ in range(muestras): # Numero de muestras a guardar
            G = np.random.choice([-1, 1], size=(k, k))  # Configuración inicial aleatoria
            configuraciones_mcmc[beta][step].append(gibbs_sampler_mcmc(G, beta, step, k).copy())  # Guarda la configuracion final
        print(f"Guardado {muestras} muestras con X = {step} para beta = {beta}")

Guardado 100 muestras con X = 10000 para beta = 0
Guardado 100 muestras con X = 10000 para beta = 0.1
Guardado 100 muestras con X = 10000 para beta = 0.2
Guardado 100 muestras con X = 10000 para beta = 0.3
Guardado 100 muestras con X = 10000 para beta = 0.4
Guardado 100 muestras con X = 10000 para beta = 0.5
Guardado 100 muestras con X = 10000 para beta = 0.6
Guardado 100 muestras con X = 10000 para beta = 0.7
Guardado 100 muestras con X = 10000 para beta = 0.8
Guardado 100 muestras con X = 10000 para beta = 0.9
Guardado 100 muestras con X = 10000 para beta = 1


### Muestras del modelo de Ising con inverso de temperatura usando Prop-Wilson

Usaremos los mismos valores usados anteriormente para hacer una comparacion mas adelante

In [46]:
def gibbs_sampler_pw(k, beta):
    # Configuraciones extremas
    G_min = -np.ones((k, k))
    G_max = np.ones((k, k))
    
    while True:
        
        for _ in range(k*k):
            
            x = random.randint(0,k-1)
            y = random.randint(0,k-1)

            # Sandwiching

            # Cambio de estado en grafo min
            delta_energia_min = calcular_diferencia_energia(G_min, k, x, y)
            aceptar_cambio(G_min, x, y, beta, delta_energia_min)

            # Cambio de estado en grafo max
            delta_energia_max = calcular_diferencia_energia(G_max, k, x, y)
            aceptar_cambio(G_max, x, y, beta, delta_energia_max)
        
        # Verifica coalecencia
        if np.array_equal(G_min, G_max):
            return G_min 

configuraciones_pw = {}

k = 10

# Loop principal
for beta in beta_list: # Beta a utilizar
    configuraciones_pw[beta] = []
    for _ in range(1): # Numero de muestras a guardar
        configuraciones_pw[beta].append(gibbs_sampler_pw(k, beta).copy())  # Guarda la configuracion final
    print(f"Guardado {muestras} muestras para beta = {beta}")



KeyboardInterrupt: 

In [40]:
pprint(configuraciones_pw)

{0: [array([[ 1.,  1., -1.],
       [-1., -1.,  1.],
       [-1., -1.,  1.]])],
 0.1: [array([[ 1., -1.,  1.],
       [-1., -1., -1.],
       [ 1.,  1.,  1.]])],
 0.2: [array([[-1.,  1., -1.],
       [ 1., -1.,  1.],
       [-1.,  1., -1.]])],
 0.3: [array([[-1.,  1., -1.],
       [ 1., -1.,  1.],
       [-1.,  1., -1.]])],
 0.4: [array([[ 1., -1.,  1.],
       [-1.,  1., -1.],
       [ 1., -1.,  1.]])],
 0.5: [array([[ 1., -1.,  1.],
       [-1.,  1., -1.],
       [ 1., -1.,  1.]])],
 0.6: [array([[ 1., -1.,  1.],
       [-1.,  1., -1.],
       [ 1., -1.,  1.]])],
 0.7: [array([[ 1., -1.,  1.],
       [-1.,  1., -1.],
       [ 1., -1.,  1.]])],
 0.8: [array([[ 1., -1.,  1.],
       [-1.,  1., -1.],
       [ 1., -1.,  1.]])],
 0.9: [array([[ 1., -1.,  1.],
       [-1.,  1., -1.],
       [ 1., -1.,  1.]])],
 1: [array([[-1.,  1., -1.],
       [ 1., -1.,  1.],
       [-1.,  1., -1.]])]}
